# Projeto 1 - Ciência dos Dados

Nome: Guilherme Fugita

Nome: Paola Friedel


Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [24]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[:;\n"'']' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [25]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Usuário\Desktop\C dados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [26]:
filename = 'Big Mac.xlsx'

In [27]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevância
0,queria tnt comer um big mac 😫,1
1,"doida pra comer um big mac, daq a pouco vou am...",1
2,vou assistir miss americana dnv enquanto almoç...,1
3,qria um big mac agr,1
4,"@juhrangel_ voltou não, um passarinho me conto...",1


In [28]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevância
0,queria tnt comer um big mac 😫,1
1,"doida pra comer um big mac, daq a pouco vou am...",1
2,vou assistir miss americana dnv enquanto almoç...,1
3,qria um big mac agr,1
4,"@juhrangel_ voltou não, um passarinho me conto...",1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Escolhemos o produto do McDonalds, o lanche BigMac, tudo que é considerado relevante agrega valores positivos e negativos para o produto como uma critica tanto quanto o desejo dos usuários em relação ao produto.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [29]:
#--------TUDO--------
with open('txts/tudo.txt', 'w', encoding="utf-8") as todos_tweets:
    for j in range(len(train)):
        tweetTr = train.loc[j,"Treinamento"]
        tweetTrC = cleanup(tweetTr)
        todos_tweets.write("{0} ".format(tweetTrC))
        
#--------RELEVANTES--------
Rel = train.loc[train['Relevância'] == 1, ['Treinamento']]

with open('txts/relevantes.txt', 'w', encoding="utf-8") as tweets_relevantes:
    for q in range(len(Rel)):
        RelTr = Rel.iloc[q, 0]
        RelTrC = cleanup(RelTr)
        tweets_relevantes.write("{0} ".format(RelTrC))

#--------TRRELEVANTES--------
IRel = train.loc[train['Relevância'] == 0, ['Treinamento']]
with open('txts/irrelevantes.txt', 'w', encoding="utf-8") as tweets_irrelevantes:
    for q in range(len(IRel)):
        IRelTr = IRel.iloc[q, 0]
        IRelTrC = cleanup(IRelTr)
        tweets_irrelevantes.write("{0} ".format(IRelTrC))
    
#--------LENDO OS ARQUIVOS--------   
with open("txts/tudo.txt", "r", encoding="utf-8") as arquivo_texto:
    tudo_raw = arquivo_texto.read()  
    
with open("txts/relevantes.txt", "r", encoding="utf-8") as arquivo_texto:
    relevantes_raw = arquivo_texto.read() 
    relevantes = relevantes_raw
with open("txts/irrelevantes.txt", "r", encoding="utf-8") as arquivo_texto:
    irrelevantes_raw = arquivo_texto.read() 
    irrelevantes = irrelevantes_raw
  



FileNotFoundError: [Errno 2] No such file or directory: 'txts/tudo.txt'

In [ ]:

#--------RELEVANTES--------
todas_palavras_relevantes = relevantes_raw.split()
serie_relevantes = pd.Series(todas_palavras_relevantes)
tabela_relevantes = serie_relevantes.value_counts()
tabela_relevantes_relativa = serie_relevantes.value_counts(True)

#--------TRRELEVANTES--------
todas_palavras_irrelevantes = irrelevantes_raw.split()
serie_irrelevantes = pd.Series(todas_palavras_irrelevantes)
tabela_irrelevantes = serie_irrelevantes.value_counts()
tabela_irrelevantes_relativa = serie_irrelevantes.value_counts(True)

#--------TUDO--------
tudo = relevantes + irrelevantes
todas_palavras = tudo.split()
serie_tudo = pd.Series(todas_palavras)
tabela_tudo_relativa = serie_tudo.value_counts(True)


In [ ]:
probR = len(todas_palavras_relevantes) / len(todas_palavras)
probI = len(todas_palavras_irrelevantes) / len(todas_palavras)

print(round(probR,5))
print(round(probI,5))


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [ ]:
#--------TESTE--------
classificadores = []
classificador = [] #para a comparação final
tweets =[]
for j in range(len(test)):
    tweetTeste = test.loc[j, "Teste"]
    tweetTeste = cleanup(tweetTeste)
    tweets.append(tweetTeste)
    classificadores.append(tweets)
    classificador.append(tweetTeste)
    tweets = []
 

In [ ]:
valores = []
for h in range(len(classificadores)):
    g = classificadores[h]
    for tweet in g:
        palavrasR = []
        for i in tweet.split():
            if i not in tabela_relevantes:
                pr = 0
                pR = (pr + 1) / (len(relevantes) + len(tudo))
                palavrasR.append(pR)
                probTweetDadoR = np.prod(palavrasR)
            else: 
                pr = tabela_relevantes[i]
                pR = (pr + 1) / (len(relevantes) + len(tudo))
                palavrasR.append(pR)
                probTweetDadoR = np.prod(palavrasR)
            
        palavrasI = []
        for j in tweet.split():
            if i not in tabela_irrelevantes:
                pi1 = 0
                pI = (pi1 + 1) / (len(irrelevantes) + len(tudo))
                palavrasI.append(pI)
                probTweetDadoI = np.prod(palavrasI)
            else: 
                pii = tabela_irrelevantes[i]
                pI = (pi1 + 1) / (len(irrelevantes) + len(tudo))
                palavrasR.append(pI)
                probTweetDadoI = np.prod(palavrasI)
            
        probRTweet = probTweetDadoR * probR
        probITweet = probTweetDadoI * probI
        if probRTweet > probITweet:
            valores.append("Relevante")
        else:
            valores.append("Irrelevante")

In [ ]:
for i in range(len(valores)):
    test.loc[i, "Comparador"] = valores[i]
test

In [ ]:
#--------VERDADEIRO POSITIVO--------
rel1x = test.loc[(test['Relevância'] == 1)]
VP = rel1x.loc[(rel1x['Comparador'] == 'Relevante')]

#--------FALSO POSITIVO--------
rel0x = test.loc[(test['Relevância'] == 0)]
FP = rel0x.loc[(rel0x['Comparador'] == 'Relevante')]

#--------VERDADEIRO NEGATIVO--------
rel0y = test.loc[(test['Relevância'] == 0)]
VN = rel0y.loc[(rel0y['Comparador'] == 'Irrelevante')]

#--------FALSO NEGATIVO--------
rel1y = test.loc[(test['Relevância'] == 1)]
FN = rel1y.loc[(rel1y['Comparador'] == 'Irrelevante')]

In [ ]:
pVP = (len(VP)*100)/500
print("PORCENTAGEM VERDADEIRO POSITIVO: {0}".format(pVP))

pFP = (len(FP)*100)/500
print("PORCENTAGEM FALSO POSITIVO: {0}".format(pFP))

pVN = (len(VN)*100)/500
print("PORCENTAGEM VERDADEIRO NEGATIVO: {0}".format(pVN))

pFN = (len(FN)*100)/500
print("PORCENTAGEM FALSO NEGATIVO: {0}".format(pFN))

In [ ]:
#------ACURACIA EM PORCENTAGEM----
acuP = ((len(VP)+len(VN))*100)/500
print("ACURÁCIA EM PORCENTAGEM: {0}".format(acuP))
#------ACURACIA EM VALORES--------
acu = ((len(VP)+len(VN)))/500
print("ACURÁCIA: {0}".format(acu))

___
### Concluindo

O comparador apresentou um desempenho razoavelmente bom, levando em conta que a porcentagem de classificações falsas está menor que 20%, sendo falsos positivos 0% e falsos negativos de 16.4%. Entretanto, esse valor ainda não é excelente para um classificador. Com isso, algumas coisas que podem ser feitas para melhorar seu desempenho são aumentar a base de
dados (pegar um numero maior de tweets) e aumentar a quantidades de classificadores, como por exemplo, adicionar uma 
classificação exclusiva para tweets que apresentam ironia e ou dupla negação (Na implementação desse classificador foi optado 
por direcionar esses tipos de tweets como relevantes). Posto isto, como o classificador apresentou uma porcentagem de acertos
maior que 80% é evidente que o projeto necessita de ajustes, porém, nada drástico. Por fim, apos essa analise de dados e do 
desempenho do projeto é valido que o investimento no projeto continue. Obrigada a todos.

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**